In [1]:
#from datetime import datetime
#import pytz
#import json
#import urllib
import numpy as np
import pandas as pd
from datetime import datetime, timedelta

from bokeh.plotting import figure, show
from bokeh.models import ColumnDataSource, LinearAxis, Range1d, Band
from bokeh.models import HoverTool, TapTool, DatetimeTickFormatter, DateFormatter
from bokeh.tile_providers import get_provider, Vendors
from scipy.spatial.distance import cdist
#from bokeh.models.renderers import GlyphRenderer
#from bokeh.models.widgets import DataTable, TableColumn, Slider, Dropdown
from bokeh.layouts import layout, column, row, widgetbox
from bokeh.io import output_notebook
output_notebook()

Loading BokehJS ...

In [2]:
base_url = 'http://meteo145.uibk.ac.at/'
station = 'innsbruck'
time = str(3)

url = base_url + station + '/' + time

In [3]:
df = pd.read_json(url)
df['time'] = [datetime(1970, 1, 1) + timedelta(milliseconds=ds) for ds in df['datumsec']]
df = df.set_index('time')
df = df.drop(columns='datumsec')
df.head()

,rr,dd,tp,p,tl,so,ff
time,,,,,,,
2019-10-21 08:50:00,0.0,235,9.8,949.1,16.3,9.450000,4.1
2019-10-21 09:00:00,0.0,235,9.9,949.3,17.1,8.950000,3.9
2019-10-21 09:10:00,0.0,290,10.3,949.0,17.6,7.350000,5.1
2019-10-21 09:20:00,0.0,273,10.2,949.2,18.0,9.233333,4.8
2019-10-21 09:30:00,0.0,257,10.2,949.1,18.4,9.100000,3.2


In [4]:
for column in df.columns:
    df[df[column] == -99.9] = np.nan


In [5]:
def merc(lat, lon):
    """Convert latitude and longitude into mercator's x and y position
    Parameters
    ----------
    lon : longitude
    lat : latitude
    """
    r_major = 6378137.000
    x = r_major * np.radians(lon)
    scale = x/lon
    y = 180.0/np.pi * np.log(np.tan(np.pi/4.0 + 
        lat * (np.pi/180.0)/2.0)) * scale
    return (x, y)

In [6]:
stations = pd.DataFrame({'lat':[47.26], 'lon':[11.45]}, index=['innsbruck'])
[stations['x'], stations['y']]= merc(stations['lat'],stations['lon'])
stations

,lat,lon,x,y
innsbruck,47.26,11.45,1.274608e+06,5.984616e+06


In [7]:
tile_provider = get_provider(Vendors.CARTODBPOSITRON)
map_tools = 'box_zoom,pan,save,hover,reset,tap,wheel_zoom'
map_plot = figure(x_range=(1108137, 1417582), y_range=(5895123 , 6088551), plot_width=600, plot_height=350,
           x_axis_type="mercator", y_axis_type="mercator", tools=map_tools)#, sizing_mode="scale_both")
map_plot.add_tile(tile_provider)
map_plot.circle(x="x", y="y", size=15, fill_color="blue", fill_alpha=0.4, source=stations);
hover_map = map_plot.select(dict(type=HoverTool))
hover_map.tooltips = [("Stationsname", "@index")]
hover_map.mode = 'mouse'
show(map_plot)

In [35]:
df['z'] = 0 #adding zero column to display band for ssd

In [46]:
def get_width():
    mindate = min(df.index)
    maxdate = max(df.index)
    return 0.8 * (maxdate-mindate).total_seconds()*1000 / len(df.index)
get_width()

478888.8888888889

In [54]:
source = ColumnDataSource(df)

p1 = figure(x_axis_type="datetime");
p1.line(x='time', y='tl', source=df);
p1.extra_y_ranges = {'ssd': Range1d(start=0, end=10)}
p1.add_layout(LinearAxis(y_range_name='ssd'), 'right')
p1.vbar(top='so', x='time', source=df, width=get_width(), fill_color='yellow', 
        line_alpha=0, line_width=0, fill_alpha=0.4, y_range_name='ssd')

show(p1)